In [78]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm


In [79]:
### Function to Commodities Data
# Define the list of commodity tickers
#tickers = ['GC=F',  # Gold futures
#           'SI=F',  # Silver futures
#           'CL=F',  # Crude oil futures
#           'HG=F',  # Copper futures
#           'NG=F']  # Natural gas futures
tickers = ['HG=F','SI=F','CL=F']

def import_commod_data(tickers, start_date):
    data = pd.DataFrame()
    if len(tickers) == 1:
        data[tickers[0]] = yf.download(tickers[0], start_date)['Adj Close']
    else:
        for t in tickers:
            data[t] = yf.download(t, start_date)['Adj Close']
    return data

start_date = '2022-01-01'
commod_data = import_commod_data(tickers, start_date)
commod_data = commod_data.reset_index() # Turn Multi-Index into Column in Pandas 
commod_data.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,HG=F,SI=F,CL=F
585,2024-05-01,4.5525,26.489000,79.000000
586,2024-05-02,4.4940,26.583000,78.949997
587,2024-05-03,4.5735,26.445000,78.110001
588,2024-05-06,4.6345,27.368999,78.480003
589,2024-05-07,4.5910,27.504999,78.320000


In [80]:
### Function to compute sigma (std dev of log normal historical stock return data)
def compute_sigma(returns):
    # Compute the standard deviation of returns
    sigma = np.std(returns) / 100
    return sigma

sigma = compute_sigma(commod_data[tickers])
print(sigma)

HG=F    0.003590
SI=F    0.021422
CL=F    0.122108
dtype: float64


In [81]:
### Function to compute Black76 Model
def Black76(option_type, r, S_0, K, T, sigma):
    # Convert S_0 and K to NumPy arrays for element-wise operations
    S_0 = np.array(S_0)
    K = np.array(K)

    # Calculate d_1 and d_2 element-wise
    d_1 = (np.log(S_0 / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d_2 = (np.log(S_0 / K) - 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))

    if option_type == 'call':
        option_value = (np.exp(-r * T)) * (S_0 * norm.cdf(d_1) - K * norm.cdf(d_2))
    elif option_type == 'put':
        option_value = (np.exp(-r * T)) * (K * norm.cdf(-d_2) - S_0 * norm.cdf(-d_1))

    return option_value
    

In [92]:
### Return Black76 Option Values
# Extract most recent commodity prices as a list
S_0 = commod_data.iloc[-1, 1:].values.tolist()
# Create Strike Price list
K = [6, 28.5, 79.5]
# Type Specification and other metrics
option_type = 'put'
r = 0.05
T = 1

# Call B76 Function
option_val = Black76(option_type, r, S_0, K, T, sigma)
#print(option_val)
# Return results in a printed list format
option_val_list = []
for i, ticker in enumerate(tickers):
    option_val_list.append((ticker, option_val[i]))

# Print each item in output on a new line
for item in option_val_list:
    print(f"{item[0]}: {round(item[1], 4)}")


HG=F: 1.382
SI=F: 2.6283
CL=F: 6.557


In [89]:
### Manual Test
S_0 = 42
K = 42
T = 1.5
r = 0.05
sigma = 0.20
option_type = 'call'

test = Black76(option_type, r, S_0, K, T, sigma)
print('The value of the put given the above inputs is: $' + str(round(test, 4)) + ' and the answer is supposed to be: $3.7982')

The value of the put given the above inputs is: $3.7982 and the answer is supposed to be: $3.7982
